In [1]:
import logging

from typing import List, Any, Optional, Dict, Tuple
from llama_index.vector_stores.types import (
    VectorStore,
    VectorStoreQuery,
    VectorStoreQueryResult,
)
import llama_index
from llama_index import StorageContext, VectorStoreIndex, ServiceContext
from llama_index.schema import TextNode, BaseNode, Document
from llama_index.memory import ChatMemoryBuffer
from tokenizers import Tokenizer

import oci
import ads

# only to check versions!
import oracledb

from oci_utils import load_oci_config
from oracle_vector_db import OracleVectorStore

# using factory methods
from prepare_chain import create_reranker, create_embedding_model, create_llm
from config import TOKENIZER

In [2]:
# version I'm using
print(f"oracledb version: {oracledb.__version__}")
print(f"oci version: {oci.__version__}")
print(f"llama_index version: {llama_index.__version__}")

oracledb version: 2.0.0.dev20231121
oci version: 2.112.1+preview.1.1649
llama_index version: 0.9.21


In [3]:
cohere_tokenizer = Tokenizer.from_pretrained(TOKENIZER)

In [4]:
# parameters

# for similarity query
TOP_K = 4
# for Reranker
TOP_N = 2

In [5]:
# Build the entire chain
oci_config = load_oci_config()

# need to do this way
api_keys_config = ads.auth.api_keys(oci_config)

# english, or for other language use: multilingual

# Embedding model
embed_model = create_embedding_model(auth=api_keys_config)

# Vector Store
v_store = OracleVectorStore(verbose=False)

# reranker
# Reranker is deployed in OCI Data Science as a Model Deployment

reranker = create_reranker(auth=api_keys_config, verbose=True)

# LLM
llm_oci = create_llm(auth=api_keys_config)

# Llama-index
service_context = ServiceContext.from_defaults(llm=llm_oci, embed_model=embed_model)

index = VectorStoreIndex.from_vector_store(
    vector_store=v_store, service_context=service_context
)

2024-01-11 21:59:34,813 - INFO - Created OCI reranker client...
2024-01-11 21:59:34,813 - INFO - Region: eu-frankfurt-1...
2024-01-11 21:59:34,813 - INFO - Deployment id: ocid1.datasciencemodeldeployment.oc1.eu-frankfurt-1.amaaaaaangencdyaulxbosgii6yajt2jdsrrvfbequkxt3mepz675uk3ui3q...
2024-01-11 21:59:34,814 - INFO - 


#### To add message_history, create a chat_engine

In [6]:
memory = ChatMemoryBuffer.from_defaults(
    token_limit=2800, tokenizer_fn=cohere_tokenizer.encode
)

chat_engine = index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    verbose=False,
    similarity_top_k=TOP_K,
    node_postprocessors=[reranker],
)

In [7]:
# start a new conversation
chat_engine.reset()

In [8]:
memory.chat_history

[]

In [9]:
response = chat_engine.chat("What is the last release for Oracle database?")

print("")
print(response.response)

2024-01-11 21:59:35,803 - INFO - Condensed question: What is the last release for Oracle database?
2024-01-11 21:59:37,408 - INFO - Reranking...
2024-01-11 21:59:37,654 - INFO - ...elapsed time: 0.25 sec.



 The last release for Oracle database is Oracle Database 23c. It was released in October 2023 and is the latest version of the Oracle Database. 

Would you like to know more about Oracle Database 23c? 


In [10]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the last release for Oracle database?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' The last release for Oracle database is Oracle Database 23c. It was released in October 2023 and is the latest version of the Oracle Database. \n\nWould you like to know more about Oracle Database 23c? ', additional_kwargs={})]

In [11]:
response = chat_engine.chat("Is it a long term release?")

print("")
print(response.response)

2024-01-11 21:59:43,144 - INFO - Condensed question:  What is the general release cycle for Oracle Database? 

This reframed question stands on its own and is directed at understanding more about the Oracle Database releases in general. 

Would you like me to tell you about the key features of Oracle Database 23c? 
2024-01-11 21:59:44,271 - INFO - Reranking...
2024-01-11 21:59:44,610 - INFO - ...elapsed time: 0.34 sec.



 Yes, Oracle Database 23c is a long-term support release. Oracle Database 23c is the next long-term support release of Oracle Database. "Long-term support" refers to the period during which Oracle supports a particular version and provides updates for it. This distinction is significant because it implies that Oracle Database 23c will receive maintenance and support from Oracle for an extended period, offering businesses and users the assurance of continued assistance and stability for their applications.

It is advisable to check Oracle's official documentation or website for the specific dates and terms of their long-term support policy to obtain precise information regarding the duration of support for Oracle Database 23c. 

Would you like to know more about the features of Oracle Database 23c? 


In [12]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the last release for Oracle database?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' The last release for Oracle database is Oracle Database 23c. It was released in October 2023 and is the latest version of the Oracle Database. \n\nWould you like to know more about Oracle Database 23c? ', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Is it a long term release?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' Yes, Oracle Database 23c is a long-term support release. Oracle Database 23c is the next long-term support release of Oracle Database. "Long-term support" refers to the period during which Oracle supports a particular version and provides updates for it. This distinction is significant because it implies that Oracle Database 23c will receive maintenance and support from Oracle for an extended period, off

In [13]:
response = chat_engine.chat("What about new features for JSON?")

print("")
print(response.response)

2024-01-11 21:59:51,039 - INFO - Condensed question:  What new features for JSON are available in the Oracle Database 23c release?
2024-01-11 21:59:52,108 - INFO - Reranking...
2024-01-11 21:59:52,361 - INFO - ...elapsed time: 0.25 sec.



 Certainly! Oracle Database 23c boasts several new features that enhance its support for JSON (JavaScript Object Notation). Here are some of the key improvements:

1. **JSON-Relational Duality**: Oracle Database 23c introduces a new feature called JSON-Relational Duality. It allows you to access and update JSON data in the database using both JSON syntax and relational SQL syntax. This flexibility enables developers to work with JSON data using the syntax that best suits their application and use case.

2. **Native Database Support for JSON**: Oracle Database 23c builds upon its native support for JSON, which was initially introduced in earlier versions. It offers features such as transactions, indexing, and declarative querying, making it a powerful choice for working with JSON data in a relational database environment.

3. **JSON Schema Validation**: Oracle Database 23c introduces JSON Schema validation, a feature that enhances the integrity and consistency of JSON data in the datab

In [14]:
memory.chat_history

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the last release for Oracle database?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' The last release for Oracle database is Oracle Database 23c. It was released in October 2023 and is the latest version of the Oracle Database. \n\nWould you like to know more about Oracle Database 23c? ', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='Is it a long term release?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content=' Yes, Oracle Database 23c is a long-term support release. Oracle Database 23c is the next long-term support release of Oracle Database. "Long-term support" refers to the period during which Oracle supports a particular version and provides updates for it. This distinction is significant because it implies that Oracle Database 23c will receive maintenance and support from Oracle for an extended period, off

In [15]:
response = chat_engine.chat("Yes give me some more details regarding the point n. 1")

print("")
print(response.response)

2024-01-11 22:00:33,000 - INFO - Condensed question:  How does JSON-Relational Duality in Oracle Database 23c enhance the flexibility for developers to work with JSON data? 

JSON-Relational Duality, as introduced in Oracle Database 23c, offers a significant advantage to developers working with JSON data in the database. This feature provides a flexible approach to access and modify JSON data using either JSON syntax or relational SQL syntax. 

Here's how it enhances flexibility:

1. **Choice of Syntactic Styles**: With JSON-Relational Duality, developers can choose to work with JSON data using the familiar SQL syntax they are accustomed to in relational databases. This allows them to leverage their existing SQL skills and expertise when working with JSON data, eliminating the need to learn a new JSON-specific querying language.

2. **Seamless Integration**: JSON-Relational Duality enables the seamless integration of JSON data with relational data within the same database. This means t

In [ ]:
memory.chat_history

In [ ]:
response = chat_engine.chat("What about the symptoms for long covid?")

print("")
print(response.response)

In [ ]:
memory.chat_history

In [ ]:
response = chat_engine.chat("Tell me more about Respiratory issues")

print(response.response)

In [ ]:
memory.chat_history

In [ ]:
response = chat_engine.chat("Whats is JSON SChema in Oracle database?")

print("")
print(response.response)

In [ ]:
response = chat_engine.chat("Could you show me an example?")

print("")
print(response.response)